# Import

In [1]:
# Misc
import os
import json
import joblib
import warnings

# Data management
import numpy as np
import pandas as pd

# Sound treatments
import librosa
import soundfile as sf
from scipy import signal

# TRILL
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()
assert tf.executing_eagerly()
import tensorflow_hub as hub

## Metrics
import tensorflow_addons as tfa
from tensorflow_addons.layers.netvlad import NetVLAD

# Plot
import matplotlib.pyplot as plt

# Environment

In [2]:
# Inactivate warnings
warnings.filterwarnings('ignore')

# Allow to display all dataframes columns
pd.set_option("display.max_columns", None)

# Display Tensorlfow version
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 2.6.0


In [3]:
#DATA_PATH = '/kaggle/input/birdclef-2022/'
#WORKING_PATH = '/kaggle/working/'
#TRILL_PATH = '/kaggle/input/ziptrill/'
#MODEL_PATH = '/kaggle/input/trillmodels/'

DATA_PATH = './data/'
WORKING_PATH = './working/kernel/'
TRILL_PATH = './working/kernel/trill/'
MODEL_PATH = './working/kernel/'

strategy = 3

# Data load

In [4]:
# Load meta data
train_meta = pd.read_csv(DATA_PATH + 'train_metadata.csv', sep=',', decimal='.', encoding='utf8', low_memory=False)

# Load scored birds
with open(DATA_PATH + 'scored_birds.json') as sbfile:
    scored_birds = json.load(sbfile)
    
# Focus on 21 scored classes
labels = list(train_meta[train_meta['primary_label'].isin(scored_birds)]['primary_label'].unique())
labels

['akiapo',
 'aniani',
 'apapan',
 'barpet',
 'crehon',
 'elepai',
 'ercfra',
 'hawama',
 'hawcre',
 'hawgoo',
 'hawhaw',
 'hawpet1',
 'houfin',
 'iiwi',
 'jabwar',
 'maupar',
 'omao',
 'puaioh',
 'skylar',
 'warwhe1',
 'yefcan']

# Modelization

## Preprocessing

In [5]:
def extractFeatures_trill(y, sr):
    # Sound noise reduction
    b, a = signal.butter(10, 1000/(sr/2), btype='highpass')
    y = signal.lfilter(b, a, y)
    # Resample
    y = librosa.resample(y, sr, 16000)

    return y

## Model

In [6]:
def create_trill():
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input((80000,)))

    trill_layer = hub.KerasLayer(
        handle=TRILL_PATH,
        trainable=False,
        arguments={'sample_rate': int(16000)},
        output_key='embedding',
        output_shape=[None, 2048]
    )

    model.add(trill_layer)
    model.add(NetVLAD(num_clusters=8))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dense(21, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(l=1e-5)))

    return model

# Submission

## Load model

In [7]:
# Model
Trill = create_trill()
Trill.load_weights(MODEL_PATH + 'finetune_da_Trill.h5')
Trill.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=[tfa.metrics.F1Score(name='f1macro', num_classes=len(labels), average='macro')])
Trill.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, None, 2048)        51964864  
_________________________________________________________________
net_vlad (NetVLAD)           (None, 16384)             32776     
_________________________________________________________________
batch_normalization (BatchNo (None, 16384)             65536     
_________________________________________________________________
dense (Dense)                (None, 256)               4194560   
_________________________________________________________________
dense_1 (Dense)              (None, 21)                5397      
Total params: 56,263,133
Trainable params: 4,265,501
Non-trainable params: 51,997,632
_________________________________________________________________


## Process

In [8]:
test_path = DATA_PATH + '/test_soundscapes/'
files = [f.split('.')[0] for f in sorted(os.listdir(test_path))]
print('Number of test soundscapes:', len(files))

Number of test soundscapes: 1


In [9]:
Class_weights = {0: 3.1081275724908664, 1: 3.7656475754076606, 2: 1.0441328733521613, 3: 3.0691726047550407,
                 4: 5.264737430672896, 5: 2.8777563180552033, 6: 5.46209686483139, 7: 1.69247564674142,
                 8: 2.124710995914189, 9: 4.432477447650233, 10: 4.419574042814324, 11: 4.854309533222602, 12: 1,
                 13: 1.2511821233781157, 14: 1.5747391996382374, 15: 4.445549529217585, 16: 1.91982088490933,
                 17: 6.173593184059532, 18: 1, 19: 1.3683585469443174, 20: 2.205399907164332}

In [10]:
if strategy == 1:
    limits = [1e-1, 1e-3, 1e-4, 1e-6, 1e-8]
elif strategy == 2:
    limits = [1e-1, 1e-3, 1e-6, 1e-7, 1e-9]
else:
    limits = [1e-1, 1e-3, 1e-6, 1e-10, 1e-12]
    
print('limits', limits)

limits [0.1, 0.001, 1e-06, 1e-10, 1e-12]


In [11]:
def testscore(prediction):   
    result = []

    for i in range(0, len(prediction)):
        if Class_weights[i] < 2:
            if prediction[i] >= limits[0]:
                result.append(prediction[i])
        elif Class_weights[i] < 3:
            if prediction[i] >= limits[1]:
                result.append(prediction[i])
        elif Class_weights[i] < 4:
            if prediction[i] >= limits[2]:
                result.append(prediction[i])
        elif Class_weights[i] < 5:
            if prediction[i] >= limits[3]:
                result.append(prediction[i])
        elif Class_weights[i] > 5:
            if prediction[i] >= limits[4]:
                result.append(prediction[i])
                
    return result

In [12]:
data = []

for f in files:
    file_path = test_path + f + '.ogg'

    # Load audio file
    audio, sr = librosa.load(file_path)

    # Get number of samples for 5 seconds
    buffer = 5 * sr
    block_min = 5 * sr

    samples_total = len(audio)
    samples_wrote = 0
    counter = 1

    while samples_wrote < samples_total:
        # check if the buffer is not exceeding total samples
        if buffer > (samples_total - samples_wrote):
            buffer = samples_total - samples_wrote

        block = audio[samples_wrote: (samples_wrote + buffer)]

        # check if last block is as long as previous ones
        if block.shape[0] < (block_min):
            listofzeros = np.array([0] * (block_min - block.shape[0]))
            block = np.hstack([block, listofzeros])

        # Features extraction
        block = extractFeatures_trill(block, sr)

        X = np.empty((1, 80000))
        X[0] = np.array(block)

        # Prediction
        pred = Trill.predict_on_batch(X)
        #print('pred', pred)

        #countOK = list(filter(lambda score: score >= 1e-6, pred[0]))
        countOK = testscore(pred[0])
        #print('countOK', countOK)

        label_indexes = []
        for i in range(0, len(countOK)):
            label_indexes.append(np.argsort(np.max(pred, axis=0))[-(i+1)])

        print(label_indexes)

        for b in scored_birds:
            segment_end = counter * 5
            row_id = f + '_' + b + '_' + str(segment_end)
            target = False
            for label_index in label_indexes:
                if labels[label_index] == b:
                    target = True
            data.append([row_id, target])
        counter += 1
        samples_wrote += buffer

submission_df = pd.DataFrame(data, columns=['row_id', 'target'])
submission_df.head(21)

[19, 14, 18, 3, 4, 17]
[19, 18, 14, 3, 4]
[19, 14, 18, 4, 3, 7, 17]
[19, 14, 18, 3, 4, 2, 13]
[19, 14, 18, 3, 7, 4, 10]
[19, 14, 18, 7, 3]
[19, 14, 18, 2, 4, 3]
[19, 14, 18, 3, 7, 4, 17]
[19, 14, 18, 3, 10, 4]
[19, 18, 14, 3, 4, 7]
[19, 14, 18, 3, 9, 2]
[19, 14, 18, 4, 7, 6, 13, 3]


,row_id,target
0,soundscape_453028782_akiapo_5,False
1,soundscape_453028782_aniani_5,False
2,soundscape_453028782_apapan_5,False
3,soundscape_453028782_barpet_5,True
4,soundscape_453028782_crehon_5,True
5,soundscape_453028782_elepai_5,False
6,soundscape_453028782_ercfra_5,False
7,soundscape_453028782_hawama_5,False
8,soundscape_453028782_hawcre_5,False
9,soundscape_453028782_hawgoo_5,False


In [13]:
submission_df.to_csv(WORKING_PATH + 'submission.csv', index=False)